In [6]:
#Loading assembly keywords and functions
%run '../pyssemble/pyInstruction.py'
#Alternative: from pyssemble.pyInstruction import *

In [9]:
py_inst = pyInstruction() #only useful for debugging later on

## Simple Assembly commands li, add, sub

In [13]:
li(a1, 5)
li(a2, 6)
add(a3, a1, a2)